# Jupyter Notebook Interactive Mode Demo
This is a scripting page that prints actual raw data in Python.

# Importing necessary libraries
1. Core Packages: dotenv, requests, httpx, and pymilvus.
2. LangChain and Extensions:
    * Langchain is used for processing language chains, and Milvus is the vector database for storing embeddings.
    * langchain: Core package.
    * langchain-core, langchain-mistralai, langchain-cohere, langchain-milvus, langchain-community, langchain-text-splitters, and langchain-huggingface for additional modules.

# Importing necessary libraries

Import the required libraries needed for the RAG

In [ ]:

# Installing dependencies if not already installed
!pip install os requests httpx pymilvus sqlite3
!pip install langchain langchain-core langchain-mistralai langchain-cohere langchain-milvus langchain-community langchain-text-splitters langchain-huggingface

import os
from dotenv import load_dotenv
from pymilvus import connections, utility
import sqlite3

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_milvus import Milvus
from langchain_community.document_loaders import WebBaseLoader, RecursiveUrlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain_huggingface import HuggingFaceEmbeddings

from requests.exceptions import HTTPError
from httpx import HTTPStatusError
import warnings

warnings.filterwarnings('ignore')

# Display to confirm successful import
print("Dependencies imported successfully.")

Dependencies imported successfully.


# Load env variables

Load the env variables needed for operation of the RAG. `CORPUS_SOURCE` can be changed to load a different corpus. `MISTRAL_API_KEY` contains the MistralAi API key. `MILVUS_URI` is the path for the milvus lite database file. `MODEL_NAME` is the embedding model used on the corpus

In [ ]:
CORPUS_SOURCE = 'https://www.csusb.edu/its'
MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")
MILVUS_URI = "milvus/jupyter_milvus_vector.db"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
def vector_store_check(uri):
    """
    Returns response on whether the milvus database exists

    Returns:
        boolean
    """
    # Create the directory if it does not exist
    head = os.path.split(uri)
    os.makedirs(head[0], exist_ok=True)
    
    # Connect to the Milvus database
    connections.connect("default",uri=uri)

    # Return True if exists, False otherwise
    return utility.has_collection("IT_support")

In [ ]:
def get_embedding_function():
    """
    Returns embedding function for the model

    Returns:
        embedding function
    """
    embedding_function = HuggingFaceEmbeddings(model_name=MODEL_NAME)
    return embedding_function

In [ ]:
def load_documents_from_web():
    """
    Load the documents from the web and store the page contents

    Returns:
        list: The documents loaded from the web
    """
    loader = RecursiveUrlLoader(
        url=CORPUS_SOURCE,
        prevent_outside=True,
        base_url=CORPUS_SOURCE
        )
    documents = loader.load()
    
    return documents

In [ ]:
def load_existing_db(uri=MILVUS_URI):
    """
    Load an existing vector store from the local Milvus database specified by the URI.

    Args:
        uri (str, optional): Path to the local milvus db. Defaults to MILVUS_URI.

    Returns:
        vector_store: The vector store created
    """
    # Load an existing vector store
    vector_store = Milvus(
        collection_name="IT_support",
        embedding_function = get_embedding_function(),
        connection_args={"uri": uri},
    )
    print("Vector Store Loaded")
    return vector_store

In [ ]:
def split_documents(documents):
    """
    Split the documents into chunks

    Args:
        documents (list): The documents to split

    Returns:
        list: list of chunks of documents
    """
    # Create a text splitter to split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=300,
        is_separator_regex=False,
    )
    
    docs = text_splitter.split_documents(documents)
    return docs

In [ ]:
def create_vector_store(docs, embeddings, uri):
    """
    This function initializes a vector store using the provided documents and embeddings.

    Args:
        docs (list): A list of documents to be stored in the vector store.
        embeddings : A function or model that generates embeddings for the documents.
        uri (str): Path to the local milvus db

    Returns:
        vector_store: The vector store created
    """
    # Create a new vector store and drop any existing one
    vector_store = Milvus.from_documents(
        documents=docs,
        embedding=embeddings,
        collection_name="IT_support",
        connection_args={"uri": uri},
        drop_old=True,
    )
    
    print("Vector Store Created")
    return vector_store

In [86]:
# Initialize a connection to the Milvus database with the specified collection name.
# "IT_support" is the collection that stores embeddings related to IT support queries.
# The `embedding_function` is a callable function that converts text to embeddings, 
# ensuring the retrieved data is in the same vector format as expected for querying.

In [88]:
def initialize_milvus(uri: str=MILVUS_URI):
    """
    Initialize the vector store for the RAG model

    Args:
        uri (str, optional): Path to the local milvus db. Defaults to MILVUS_URI.

    Returns:
        vector_store: The vector store created
    """
    if vector_store_check(uri):
        vector_store = load_existing_db(uri)
        print("Embeddings loaded from existing Database")
    else:
        embeddings = get_embedding_function()
        print("Embeddings Loaded")
        documents = load_documents_from_web()
        print("Documents Loaded")
        print(len(documents))
    
        # Split the documents into chunks
        docs = split_documents(documents=documents)
        print("Documents Splitting completed")
    
        vector_store = create_vector_store(docs, embeddings, uri)

    return vector_store

initialize_milvus()

In [84]:
# Define the template for the prompt. This template provides the format and guidelines the AI assistant 
# should follow when generating responses to user questions. The assistant is instructed to use only 
# the information within the <context> tags, keeping answers fact-based and concise.
# Create an instance of PromptTemplate using the defined template. This instance enables the AI assistant 
# to format responses according to the structure defined in PROMPT_TEMPLATE. The prompt accepts two input 
# variables, 'context' and 'question', which are dynamically filled in when the prompt is used in a query.

In [82]:
def create_prompt():
    """
    Create a prompt template for the RAG model

    Returns:
        PromptTemplate: The prompt template for the RAG model
    """
    # Define the prompt template
    PROMPT_TEMPLATE = """
    Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
    Only use the information provided in the <context> tags.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    <question>
    {input}
    </question>

    The response should be specific and use statistics or numbers when possible.

    Assistant:"""

    # Create a PromptTemplate instance with the defined template and input variables
    prompt = PromptTemplate(
        template=PROMPT_TEMPLATE, input_variables=["context", "question"]
    )
    print("Prompt Created")

    # Return the created prompt template to be used with the RAG model
    return prompt

print(create_prompt())

Prompt Created
input_variables=['context', 'input'] template="\n    Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.\n    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.\n    Only use the information provided in the <context> tags.\n    If you don't know the answer, just say that you don't know, don't try to make up an answer.\n    <context>\n    {context}\n    </context>\n\n    <question>\n    {input}\n    </question>\n\n    The response should be specific and use statistics or numbers when possible.\n\n    Assistant:"


In [ ]:
# This function sets up and runs a RAG model pipeline, integrating the following components:

#     Model Initialization: Loads the ChatMistralAI model, which serves as the foundation for generating answers.
#     Prompt Creation: Creates a prompt template for the model.
#     Vector Store Loading: Loads the Milvus vector store to retrieve relevant document data.
#     Retriever Setup: Configures a retriever to find and return relevant documents from the vector store.
#     Document Chain and Retrieval Chain Creation: Sets up the chains that guide the RAG model through document processing and response generation.
# Workflow
#     Generate Response: The RAG model is invoked with the user query, generating an answer by leveraging the relevant documents.
#     Error Handling: Handles any HTTP errors, including rate limits, by returning appropriate messages.
#     Source Aggregation: Gathers and lists up to 4 unique sources relevant to the response, adding them as citations.
#     Response Return: The function returns the generated response, enriched with cited sources.
#     Arguments
#         query (str): The input query string from the user.
#     Returns
#         str: The generated answer to the query, including sources if available.

In [90]:
def query_rag(query):
    """
    Entry point for the RAG model to generate an answer to a given query

    This function initializes the RAG model, sets up the necessary components such as the prompt template, vector store, 
    retriever, document chain, and retrieval chain, and then generates a response to the provided query.

    Args:
        query (str): The query string for which an answer is to be generated.
    
    Returns:
        str: The answer to the query
    """
    # Define the model
    model = ChatMistralAI(model='open-mistral-7b')
    print("Model Loaded")

    prompt = create_prompt()

    # Load the vector store and create the retriever
    vector_store = load_existing_db(uri=MILVUS_URI)
    retriever = vector_store.as_retriever()
    try:
        document_chain = create_stuff_documents_chain(model, prompt)
        print("Document Chain Created")

        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        print("Retrieval Chain Created")
    
        # Generate a response to the query
        response = retrieval_chain.invoke({"input": f"{query}"})
    except HTTPStatusError as e:
        print(f"HTTPStatusError: {e}")
        if e.response.status_code == 429:
            return "I am currently experiencing high traffic. Please try again later.", []
        return "I am unable to answer this question at the moment. Please try again later.", []
    
    # logic to add sources to the response
    max_relevant_sources = 4 # number of sources at most to be added to the response
    all_sources = ""
    sources = []
    count = 1
    for i in range(max_relevant_sources):
        try:
            source = response["context"][i].metadata["source"]
            # check if the source is already added to the list
            if source not in sources:
                sources.append(source)
                all_sources += f"[Source {count}]({source}), "
                count += 1
        except IndexError: # if there are no more sources to add
            break
    all_sources = all_sources[:-2] # remove the last comma and space
    response["answer"] += f"\n\nSources: {all_sources}"
    print("------------------------------------------------------------------------")
    print("Response Generated:")
    
    return response["answer"]

response = query_rag("how to connect wifi")
print(response)